# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Test set features


In [2]:
!tar xzf /content/drive/MyDrive/stat453/test2015.tar.gz

In [25]:
!ls data/vqa/feats/test2015/ | wc -l

81434


In [ ]:
!git clone https://github.com/zhaoyi3264/openvqa.git

In [5]:
!mv test2015/ openvqa/data/vqa/feats/
!mkdir openvqa/data/vqa/feats/train2014
!mkdir openvqa/data/vqa/feats/val2014

## Raw data

In [14]:
!mv *.json openvqa/data/vqa/raw/

## Pre-trained model (BAN-8)

In [ ]:
!wget https://awma1-my.sharepoint.com/:u:/g/personal/yuz_l0_tn/EbJgyL7FPTFAqzMm3HB1xDIBjXpWygOoXrdnDZKEIu34rg?download=1 -O epoch13.pkl

In [21]:
!mkdir openvqa/ckpts/ckpt_ban_8/
!mv ban_8.pkl openvqa/ckpts/ckpt_ban_8/

## Spacy model

In [ ]:
# !wget https://github.com/explosion/spacy-models/releases/download/en_vectors_web_lg-2.1.0/en_vectors_web_lg-2.1.0.tar.gz -O en_vectors_web_lg-2.1.0.tar.gz
# !pip install -q en_vectors_web_lg-2.1.0.tar.gz

In [ ]:
# %load_ext autoreload
# %autoreload 2

## Source code

https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/openvqa/core/base_cfgs.py
https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/utils/exec.py#L36
https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/utils/test_engine.py#L49
https://github.com/MILVLG/openvqa/blob/6b9bfeb2e6462b946d7e7866ffc49dd7a8bcece3/openvqa/datasets/vqa/vqa_loader.py#L68

# Load dataset

In [15]:
%cd /content/openvqa/

/content/openvqa


In [2]:
import gc
import json

import numpy as np
import torch
import torch.utils.data as Data
import yaml
from yaml import CLoader

from openvqa.datasets.dataset_loader import DatasetLoader, EvalLoader
from openvqa.datasets.vqa.eval.vqa import VQA
from openvqa.datasets.vqa.eval.vqaEval import VQAEval
from openvqa.models.model_loader import CfgLoader, ModelLoader
from run import create_parser
from utils.test_engine import test_engine

parser = create_parser()
arg = '--RUN test --MODEL ban_8 --DATASET vqa --NW 2 --CKPT_V ban_8 --CKPT_E 13'
args = parser.parse_args(arg.split(' '))

cfg_file = "configs/{}/{}.yml".format(args.DATASET, args.MODEL)
with open(cfg_file, 'r') as f:
    yaml_dict = yaml.load(f, Loader=CLoader)

__C = CfgLoader(yaml_dict['MODEL_USE']).load()
args = __C.str_to_bool(args)
args_dict = __C.parse_to_dict(args)

args_dict = {**yaml_dict, **args_dict}
__C.add_args(args_dict)
__C.proc()

print('Hyper Parameters:')
print(__C)

Checking dataset ........
Finished!

Hyper Parameters:
{ BATCH_SIZE        }->512
{ BA_HIDDEN_SIZE    }->3072
{ BBOX_NORMALIZE    }->False
{ CACHE_PATH        }->./results/cache
{ CKPTS_PATH        }->./ckpts
{ CKPT_EPOCH        }->13
{ CKPT_PATH         }->None
{ CKPT_VERSION      }->ban_8
{ CLASSIFER_DROPOUT_R }->0.5
{ DATASET           }->vqa
{ DATA_PATH         }->{'vqa': './data/vqa', 'gqa': './data/gqa', 'clevr': './data/clevr'}
{ DATA_ROOT         }->./data
{ DEVICES           }->[0]
{ DROPOUT_R         }->0.2
{ EVAL_BATCH_SIZE   }->32
{ EVAL_EVERY_EPOCH  }->False
{ FEATS_PATH        }->{'vqa': {'train': './data/vqa/feats/train2014', 'val': './data/vqa/feats/val2014', 'test': './data/vqa/feats/test2015'}, 'gqa': {'default-frcn': './data/gqa/feats/gqa-frcn', 'default-grid': './data/gqa/feats/gqa-grid'}, 'clevr': {'train': './data/clevr/feats/train', 'val': './data/clevr/feats/val', 'test': './data/clevr/feats/test'}}
{ FEAT_SIZE         }->{'vqa': {'FRCN_FEAT_SIZE': (100, 2048), 

In [3]:
dataset = DatasetLoader(__C).DataSet()

data_size = dataset.data_size
token_size = dataset.token_size
ans_size = dataset.ans_size
pretrained_emb = dataset.pretrained_emb

 ========== Dataset size: 447793
 ========== Question token vocab size: 20573
 ========== Answer token vocab size (occur more than 8 times): 3129
Finished!



In [4]:
net = ModelLoader(__C).Net(
    __C,
    pretrained_emb,
    token_size,
    ans_size
)

# net.cuda()
net.eval()

with open('ckpts/ckpt_ban_8/epoch13.pkl', 'rb') as f:
    state = torch.load(f, map_location='cpu')
net.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [5]:
dataloader = Data.DataLoader(
    dataset,
    batch_size=__C.EVAL_BATCH_SIZE,
    shuffle=False,
    num_workers=__C.NUM_WORKERS,
    pin_memory=__C.PIN_MEM
)

In [49]:
atts = []
i = 0
for frcn_feat, grid_feat, bbox_feat, ques_ix, ans in dataloader:
    lang_feat = net.embedding(ques_ix)
    lang_feat, _ = net.rnn(lang_feat)

    img_feat, _ = net.adapter(frcn_feat, grid_feat, bbox_feat)
    
    v, q = img_feat, lang_feat
    print('v.shape:\t', v.shape)
    print('q.shape:\t', q.shape)
    
    att, logits = net.backbone.BiAtt(v, q)
    print('att.shape:\t', att.shape)
    
    atts.append(att.detach().numpy().sum(axis=-1).mean(axis=0))
    i += 1
    if i > 3:
        break

v.shape:	 torch.Size([32, 100, 2048])
q.shape:	 torch.Size([32, 14, 1024])
att.shape:	 torch.Size([32, 8, 100, 14])
v.shape:	 torch.Size([32, 100, 2048])
q.shape:	 torch.Size([32, 14, 1024])
att.shape:	 torch.Size([32, 8, 100, 14])
v.shape:	 torch.Size([32, 100, 2048])
q.shape:	 torch.Size([32, 14, 1024])
att.shape:	 torch.Size([32, 8, 100, 14])
v.shape:	 torch.Size([32, 100, 2048])
q.shape:	 torch.Size([32, 14, 1024])
att.shape:	 torch.Size([32, 8, 100, 14])


In [50]:
res = np.concatenate(atts)
res.shape

(32, 100)

In [59]:
res.nbytes / 1024 / 1024 * 13993 / 3

56.937662760416664

In [52]:
mat = att[0].detach().numpy()

In [53]:
mat.shape

(8, 100, 14)

In [54]:
mat.sum(axis=-1).mean(axis=0).shape

(100,)

# Prediction (on test set)

In [35]:
%%time
ans_ix_list = []
pred_list = []

for step, (
        frcn_feat_iter,
        grid_feat_iter,
        bbox_Predictionfeat_iter,
        ques_ix_iter,
        ans_iter
    ) in enumerate(dataloader):
    print("\rEvaluation: [step %4d/%4d]" % (
        step,
        int(data_size / __C.EVAL_BATCH_SIZE),
    ), end='          ')

    frcn_feat_iter = frcn_feat_iter.cuda()
    grid_feat_iter = grid_feat_iter.cuda()
    bbox_feat_iter = bbox_feat_iter.cuda()
    ques_ix_iter = ques_ix_iter.cuda()

    pred = net(
        frcn_feat_iter,
        grid_feat_iter,
        bbox_feat_iter,
        ques_ix_iter
    )

    pred_np = pred.cpu().data.numpy()
    pred_argmax = np.argmax(pred_np, axis=1)
    
    if pred_argmax.shape[0] != __C.EVAL_BATCH_SIZE:
        pred_argmax = np.pad(
            pred_argmax,
            (0, __C.EVAL_BATCH_SIZE - pred_argmax.shape[0]),
            mode='constant',
            constant_values=-1
        )

    ans_ix_list.append(pred_argmax)
    
    if __C.TEST_SAVE_PRED:
        if pred_np.shape[0] != __C.EVAL_BATCH_SIZE:
            pred_np = np.pad(
                pred_np,
                ((0, __C.EVAL_BATCH_SIZE - pred_np.shape[0]), (0, 0)),
                mode='constant',
                constant_values=-1
            )

        pred_list.append(pred_np)

# Evaluation: [step 6698/6698]
# CPU times: user 14h 57min 55s, sys: 19min 55s, total: 15h 17min 51s
# Wall time: 7h 40min 34s
# Evaluation: [step 13993/13993]
# CPU times: user 13min 48s, sys: 8min 59s, total: 22min 47s
# Wall time: 35min 22s

Evaluation: [step 13993/13993]          CPU times: user 13min 48s, sys: 8min 59s, total: 22min 47s
Wall time: 35min 22s


In [36]:
ans_ix_list = np.array(ans_ix_list).reshape(-1)

qid_list = [ques['question_id'] for ques in dataset.ques_list]
ans_size = dataset.ans_size

result = [{
        'answer': dataset.ix_to_ans[str(ans_ix_list[qix])],
        'question_id': int(qid_list[qix])
    } for qix in range(qid_list.__len__())]

result_eval_file = __C.CACHE_PATH + '/result_run_' + __C.VERSION
log_file = __C.LOG_PATH + '/log_run_' + __C.VERSION + '.txt'
result_eval_file += '.json'
print('Save the result to file: {}'.format(result_eval_file))
with open(result_eval_file, 'w') as f:
    json.dump(result, f)

Save the result to file: ./results/cache/result_run_6911998.json


# Evaluation (on validation set)

In [ ]:
result_eval_file = __C.CACHE_PATH + '/result_run_' + __C.VERSION
result_eval_file += '.json'

ques_file_path = __C.RAW_PATH[__C.DATASET][__C.SPLIT['val']]
ans_file_path = __C.RAW_PATH[__C.DATASET][__C.SPLIT['val'] + '-anno']
# uncomment line 165-166 in http://localhost:8888/edit/openvqa/openvqa/datasets/vqa/eval/vqa.py
vqa = VQA(ans_file_path, ques_file_path)
vqaRes = vqa.loadRes(result_eval_file, ques_file_path)

loading VQA annotations and questions into memory...
0:00:05.052310
creating index...
index created!
Loading and preparing results...     
DONE (t=0.46s)
creating index...
index created!


In [ ]:
vqaEval = VQAEval(vqa, vqaRes, n=2)

# quesIds = [d['question_id'] for d in result]
# vqaEval.evaluate(quesIds)
vqaEval.evaluate()

computing accuracy
Finished Percent: [####################] 99% Done computing accuracy


In [ ]:
vqaEval.accuracy

{'overall': 84.09,
 'perQuestionType': {'none of the above': 82.19,
  'what are the': 75.57,
  'what is': 75.66,
  'what': 69.0,
  'is this a': 97.88,
  'is this': 97.46,
  'what is the man': 84.12,
  'how many': 75.92,
  'what does the': 48.29,
  'why': 46.35,
  'is it': 98.84,
  'why is the': 47.41,
  'what color is the': 92.15,
  'is there a': 98.67,
  'is the': 96.79,
  'is that a': 97.79,
  'are these': 96.65,
  'are the': 96.26,
  'what is the': 75.58,
  'which': 68.28,
  'could': 98.61,
  'are there': 98.21,
  'what kind of': 76.9,
  'has': 97.16,
  'what color are the': 91.07,
  'are there any': 99.08,
  'is this person': 98.05,
  'does the': 97.25,
  'where is the': 60.6,
  'how many people are': 74.58,
  'can you': 97.58,
  'what type of': 77.29,
  'what is the color of the': 92.4,
  'what is on the': 76.11,
  'does this': 97.48,
  'is this an': 97.49,
  'who is': 63.25,
  'what is this': 84.76,
  'is there': 98.29,
  'are': 95.61,
  'what time': 45.95,
  'how': 54.52,
  'wha